<a href="https://colab.research.google.com/github/srkhvx/Intelligent-Cyber-Security/blob/master/Copy_of_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

- If you are using Google Colab, you can simply install PySyft using the following command:
`! pip install syft`

- If you are using PySyft locally, the easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [4]:
import syft

ModuleNotFoundError: ignored

In [5]:
! pip install syft

     |████████████████████████████████| 327kB 2.9MB/s 
     |████████████████████████████████| 256kB 47.2MB/s 
     |████████████████████████████████| 399kB 47.0MB/s 
     |████████████████████████████████| 81kB 31.7MB/s 
     |████████████████████████████████| 2.6MB 33.1MB/s 
     |████████████████████████████████| 460kB 38.4MB/s 
     |████████████████████████████████| 676.9MB 30kB/s 
     |████████████████████████████████| 204kB 46.0MB/s 
     |████████████████████████████████| 2.7MB 37.6MB/s 
     |████████████████████████████████| 266kB 57.0MB/s 
     |████████████████████████████████| 51kB 21.8MB/s 
     |████████████████████████████████| 122kB 54.9MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076234 sha256=713f8824a4b4baaf008102b8caf1452f5070085b1f283f9506521ec2cf31722d
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-li

In [6]:
import torch as th
import syft as sy

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'


In [0]:
hook = sy.TorchHook(th)  # this line of code creates an instance of th with its backend APIs modified with Syft functions

In [8]:
x = th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same
x

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [10]:
print(f'Type: {type(bob._objects)} \nValue: {bob._objects}')

Type: <class 'dict'> 
Value: {}


In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x =  x.send(bob) # send this data to bob

In [13]:
bob._objects

{24315853678: tensor([1, 2, 3, 4, 5])}

In [14]:
# What's the type of the pointer? and wht's the reason behind this type?
# What's its value?

print(f'Type: {type(x)} \nValue: {x}')

Type: <class 'torch.Tensor'> 
Value: (Wrapper)>[PointerTensor | me:99549445322 -> bob:24315853678]


In [15]:
x.location # where the Tensor located?

<VirtualWorker id:bob #objects:1>

In [16]:
x.id # the pointer ID at our machine

99549445322

In [18]:
x.id_at_location # the ID of the tensor at the remote worker

24315853678

In [19]:
x.owner

<VirtualWorker id:me #objects:0>

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id='alice')

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2,3,4])
x = x.send(bob, alice)

In [27]:
# notice what does the send fucntion on two workers return; a multi-pointer
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:35714518373 -> bob:16081551935]
	-> [PointerTensor | me:53339169396 -> alice:81045640244]

In [28]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
print(f'Type: {type(x.child)} \nValue: {x.child}')
print(f'Type: {type(x.child.child)} \nValue: {x.child.child}')

Type: <class 'syft.generic.pointers.multi_pointer.MultiPointerTensor'> 
Value: [MultiPointerTensor]
	-> [PointerTensor | me:35714518373 -> bob:16081551935]
	-> [PointerTensor | me:53339169396 -> alice:81045640244]
Type: <class 'dict'> 
Value: {'bob': [PointerTensor | me:35714518373 -> bob:16081551935], 'alice': [PointerTensor | me:53339169396 -> alice:81045640244]}


In [29]:
bob._objects

{16081551935: tensor([2, 3, 4])}

In [30]:
alice._objects

{81045640244: tensor([2, 3, 4])}

In [31]:
# try the .get() on the pointer
x = x.get()
x

[tensor([2, 3, 4]), tensor([2, 3, 4])]

In [32]:
alice._objects

{}

In [33]:
bob._objects

{}

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2, 5, 4])
x = x.send(bob, alice)

In [35]:
# try the .get(sum_results=True) on your pointer
sum = x.get(sum_results=True)
sum

tensor([ 4, 10,  8])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [37]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:236382724 -> bob:84547010087]

In [38]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:78978484487 -> bob:90001511761]

In [39]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:79981299655 -> bob:74087050039]

In [40]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [42]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [43]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

<VirtualWorker id:bob #objects:4>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1., 1],[0.5, 1],[1, 0],[0, 0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [46]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:35460118960 -> bob:91663023052]

In [47]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()

  weights.data.sub_(weights.grad * lr)
  
  weights.grad *= 0

  print(loss.get().data)

tensor(0.4827)
tensor(0.3459)
tensor(0.2529)
tensor(0.1893)
tensor(0.1455)
tensor(0.1150)
tensor(0.0935)
tensor(0.0781)
tensor(0.0668)
tensor(0.0584)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [49]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{28998618616: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [53]:
bob._objects

{}

In [54]:
x.child.garbage_collect_data  # True by default

NameError: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [56]:
bob._objects

{97503207535: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [58]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [60]:
x

(Wrapper)>[PointerTensor | me:59982624670 -> bob:74746765453]

In [61]:
bob._objects

{74746765453: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [63]:
bob._objects  # some error from Jupyter is casuing this to appear!

{74746765453: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [65]:
bob._objects

{74746765453: tensor([1, 2, 3, 4, 5])}

In [66]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [68]:
bob._objects # notice that there is only a single tensor in bob

{97206636755: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [72]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [77]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).mean()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.4161)
tensor(1.7170)
tensor(0.9163)
tensor(0.5339)
tensor(0.3467)
tensor(0.2509)
tensor(0.1985)
tensor(0.1669)
tensor(0.1456)
tensor(0.1297)


# nothing was federated up to this point!

In [78]:
# let's reeat the previous experiment in a FL approach:

import torch as th
import syft as sy

hook = sy.TorchHook(th)  

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

bob = bob.clear_objects()
alice = alice.clear_objects()

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [80]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:48935714320 -> bob:35015255834],
  (Wrapper)>[PointerTensor | me:43515366403 -> bob:53219714878]),
 ((Wrapper)>[PointerTensor | me:86607271187 -> alice:55846418054],
  (Wrapper)>[PointerTensor | me:60652030025 -> alice:43836768765])]

In [0]:
def train(iterations=3):

    model = nn.Linear(2,1) # our model created locally
    opt = optim.SGD(params=model.parameters(), lr=0.1) 

    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            # 1 zero the gradients
            opt.zero_grad()
            
            # 2 calculate predictions
            print("THIS IS DATA: ", _data)
            pred = model(_data)

            # 3 calculate loss -- MSE
            loss = ((pred - _target)**2).mean()
            # 4 calculate gradeints
            loss.backward()
            # 5 update weights
            opt.step()
            # return the model to the local machine
            model=model.get()   
            
            
        print(loss.get())

In [134]:
train(10)

THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:35870703119 -> bob:54043377362]
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:92049340162 -> alice:53394793582]
tensor(0.3769, requires_grad=True)
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:35870703119 -> bob:54043377362]
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:92049340162 -> alice:53394793582]
tensor(0.2443, requires_grad=True)
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:35870703119 -> bob:54043377362]
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:92049340162 -> alice:53394793582]
tensor(0.1746, requires_grad=True)
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:35870703119 -> bob:54043377362]
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:92049340162 -> alice:53394793582]
tensor(0.1318, requires_grad=True)
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:35870703119 -> bob:54043377362]
THIS IS DATA BOB:  (Wrapper)>[PointerTensor | me:92049340162 -> alice:53394793582]
tensor(0.1023, requires_grad=True)
THIS 

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [83]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [86]:
bob._objects

{30002250903: tensor([1, 2, 3, 4, 5])}

In [87]:
alice._objects

{52186712362: (Wrapper)>[PointerTensor | alice:52186712362 -> bob:30002250903]}

In [0]:
y = x + x

In [89]:
y

(Wrapper)>[PointerTensor | me:93817667287 -> alice:25711521498]

In [90]:
bob._objects

{6618199660: tensor([ 2,  4,  6,  8, 10]),
 30002250903: tensor([1, 2, 3, 4, 5])}

In [91]:
alice._objects

{19677958202: (Wrapper)>[PointerTensor | alice:25711521498 -> bob:6618199660],
 25711521498: (Wrapper)>[PointerTensor | alice:25711521498 -> bob:6618199660],
 52186712362: (Wrapper)>[PointerTensor | alice:52186712362 -> bob:30002250903]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [93]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [95]:
z = x + y

TensorsNotCollocatedException: ignored

In [96]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:82709693277 -> bob:36777931334]

In [97]:
bob._objects

{36777931334: tensor([1, 2, 3, 4, 5]), 39343625120: tensor([1, 2, 3, 4, 5])}

In [98]:
alice._objects

{}

In [99]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [100]:
bob._objects

{39343625120: tensor([1, 2, 3, 4, 5])}

# Lesson: Pointer Chain Operations

In [101]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [103]:
bob._objects

{8226768038: tensor([1, 2, 3, 4, 5])}

In [104]:
alice._objects

{}

In [105]:
x.move(alice)

(Wrapper)>[PointerTensor | me:55532355156 -> alice:25642019950]

In [106]:
x

(Wrapper)>[PointerTensor | me:55532355156 -> alice:25642019950]

In [107]:
bob._objects

{}

In [108]:
alice._objects

{25642019950: tensor([1, 2, 3, 4, 5])}

In [109]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [111]:
bob._objects

{88125016376: tensor([1, 2, 3, 4, 5])}

In [112]:
alice._objects

{67806947999: (Wrapper)>[PointerTensor | alice:67806947999 -> bob:88125016376]}

In [113]:
x

(Wrapper)>[PointerTensor | me:83589334662 -> alice:67806947999]

In [114]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:83589334662 -> alice:67806947999]

In [115]:
bob._objects

{}

In [116]:
alice._objects

{67806947999: tensor([1, 2, 3, 4, 5])}

In [117]:
x.move(bob)

(Wrapper)>[PointerTensor | me:2774241039 -> bob:83589334662]

In [118]:
x

(Wrapper)>[PointerTensor | me:2774241039 -> bob:83589334662]

In [119]:
bob._objects

{83589334662: tensor([1, 2, 3, 4, 5])}

In [120]:
alice._objects

{}


**Exercise:**

To avoid exposing gradients among participants, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
import syft as sy
import torch as th
from torch import nn, optim

In [0]:
# create workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [123]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=False).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=False).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=False).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=False).send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [224]:
# Send copies of the linear model to alice and bob
model_bob = model.copy().send(bob)
model_alice = model.copy().send(alice)
model.send(secureWorker)

Linear(in_features=2, out_features=1, bias=True)

In [0]:
# create two opimizers for Alice and Bob
opt_bob = optim.SGD(params=model_bob.parameters(), lr=0.1) 
opt_alice = optim.SGD(params=model_alice.parameters(), lr=0.1) 

In [226]:

# train the models at alice and bob inside one loop
for iter in range(2):
  opt_bob.zero_grad()
  opt_alice.zero_grad()
  # 2 calculate predictions
  pred_bob = model_bob(data_bob)
  pred_alice=model_alice(data_alice)
  # 3 calculate loss -- MSE
  loss_bob = ((pred_bob - target_bob)**2).mean()
  loss_alice = ((pred_alice - target_alice)**2).mean()

  # 4 calculate gradeints
  loss_bob.backward()
  loss_alice.backward()
  # 5 update weights
  opt_bob.step()
  opt_alice.step()
  # return the model to the local machine
  print("LOSS FOR BOB: ",loss_bob.get())
  print("LOSS FOR ALICE: ",loss_alice.get())


LOSS FOR BOB:  tensor(3.5588, requires_grad=True)
LOSS FOR ALICE:  tensor(0.2320, requires_grad=True)
LOSS FOR BOB:  tensor(1.0619, requires_grad=True)
LOSS FOR ALICE:  tensor(0.1264, requires_grad=True)


In [0]:

# move the models to the third worker 
model_bob.move(secureWorker)
model_alice.move(secureWorker)
# aggreegate the models (their average)
# --- use model.weight.data to access the weights, and model.bias.data to access bias
# --- use model.weight.set_(new_weights) to update the weights
# --- use model.bias.set_(new_bias) to update the bias

In [228]:
weights=((model_bob.weight.data+model_alice.weight.data)/2)
biases=((model_bob.bias.data+model_alice.bias.data)/2)
model.weight.set_(weights)
model.bias.set_(biases)

RuntimeError: ignored

In [0]:
model.weight.data=weights
model.bias.data=biases

In [0]:
datax=th.tensor([[1.,1],[0,1]], requires_grad=False)

In [231]:
prediction=model(datax)

IndexError: ignored

In [0]:
model=model.get()

In [233]:
model.owner

<VirtualWorker id:me #objects:0>